In [1]:
import ray
import ray.tune as tune

Couldn't import TensorFlow - this disables TensorBoard logging.
Couldn't import `requests` library. Be sure to install it on the client side.


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms

In [3]:
class ConvNet(nn.Module):
    """
    ConvNet from TensorFlow CNN MNIST tutorial
    (see: https://www.tensorflow.org/tutorials/layers#building_the_cnn_mnist_classifier)

    Reproduced, not optmized.
    """
    def __init__(self):
        super().__init__()
        self.block1 = self.conv_block(1, 32)
        self.block2 = self.conv_block(32, 64)
        self.hidden = nn.Linear(4 * 4 * 64, 1024)
        self.dropout = nn.Dropout(.4)
        self.out = nn.Linear(1024, 10)

    @classmethod
    def conv_block(cls, in_channels, out_channels):
        """A single convolutional block"""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
            )

    @classmethod
    def flatten(cls, x):
        """Flattens the input x to a 2d tensor"""
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return x.view(-1, num_features)

    def forward(self, x):
        """
        Define a forward pass through the network.
        
        Note: Since we're inheriting from nn.Module, this will take care of the backward
        pass and parameter update step when we use an optimizer from `torch.optim`.
        """
        # make each image 3d for compatibility with convolutional blocks
        x = x.view(-1, 1, 28, 28)
        x = self.block1(x)
        x = self.block2(x)
        x = self.flatten(x) # flatten output of convolutional section
        x = self.dropout(self.hidden(x))
        return self.out(x)


In [4]:
import os
os.getcwd()

'/home/barton/Workbench/learn/default-risk/develop'

In [5]:
import sys

In [6]:
sys.executable

'/home/barton/.anaconda/envs/default-risk/bin/python'

In [7]:
from prepare_data import prepare_data

In [8]:
train_loader, _, test_loader = prepare_data()

In [9]:
device = torch.device("cuda")


In [23]:
 def train_func():
    for epoch in range(10):
        model = ConvNet().to(device)
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=0.001)
        
        model.train()
        
        for idx, (img, label) in enumerate(train_loader):
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()
            output = model(img)
            loss = F.cross_entropy(output, label)
            loss.backward()
            optimizer.step()

In [2]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:48168 to respond...
Waiting for redis server at 127.0.0.1:46945 to respond...
Starting local scheduler with the following resources: {'CPU': 8, 'GPU': 1}.

View the web UI at http://localhost:8891/notebooks/ray_ui70849.ipynb?token=5450ca2564ba3be672777669ebb8cdb7576fc0dc3e74cf58



{'local_scheduler_socket_names': ['/tmp/scheduler52470734'],
 'node_ip_address': '172.16.23.251',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store71049496', manager_name='/tmp/plasma_manager4114304', manager_port=51047)],
 'raylet_socket_names': [],
 'redis_address': '172.16.23.251:48168',
 'webui_url': 'http://localhost:8891/notebooks/ray_ui70849.ipynb?token=5450ca2564ba3be672777669ebb8cdb7576fc0dc3e74cf58'}

In [3]:
def train_func(config, reporter):
    vals = list(range(11))
    for idx,dummy in enumerate(vals):
        reporter(timesteps_total=idx, mean_accuracy=config['dummy'])

In [4]:
tune.register_trainable("train_func", train_func)

In [6]:
all_trials = tune.run_experiments({
    "awesome": {
        "run": "train_func",
        "stop": {"mean_accuracy": 9},
        "config": {
            "dummy": tune.grid_search(list(range(11)))
        }
    }
})

== Status ==
Using FIFO scheduling algorithm.
Result logdir: /home/barton/ray_results/awesome
PENDING trials:
 - train_func_0_dummy=0:	PENDING
 - train_func_10_dummy=10:	PENDING
 - train_func_1_dummy=1:	PENDING
 - train_func_2_dummy=2:	PENDING
 - train_func_3_dummy=3:	PENDING
 - train_func_4_dummy=4:	PENDING
 - train_func_5_dummy=5:	PENDING
 - train_func_6_dummy=6:	PENDING
 - train_func_7_dummy=7:	PENDING
 - train_func_8_dummy=8:	PENDING
 - train_func_9_dummy=9:	PENDING

TF not installed - cannot log with <class 'ray.tune.logger._TFLogger'>...
Created LogSyncer for /home/barton/ray_results/awesome/train_func_0_dummy=0_2018-07-24_17-53-49tmwbs24r -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/8 CPUs, 0/1 GPUs
Result logdir: /home/barton/ray_results/awesome
PENDING trials:
 - train_func_10_dummy=10:	PENDING
 - train_func_1_dummy=1:	PENDING
 - train_func_2_dummy=2:	PENDING
 - train_func_3_dummy=3:	PENDING
 - train_func_4_dummy=4:	PENDING
 - train_func_5_dummy=5:	

TrainingResult for train_func_5_dummy=5:
  date: 2018-07-24_17-53-51
  done: true
  experiment_id: 341e3f8f8bfc4a7abe3b54a0ed198e51
  hostname: build
  mean_accuracy: 5
  node_ip: 172.16.23.251
  pid: 11215
  time_this_iter_s: 1.001103401184082
  time_total_s: 2.0022459030151367
  timestamp: 1532476431
  timesteps_this_iter: 0
  timesteps_total: 10
  training_iteration: 2
  
TrainingResult for train_func_8_dummy=8:
  date: 2018-07-24_17-53-53
  experiment_id: 11875ee85e0242e7bd6d9f4826421608
  hostname: build
  mean_accuracy: 8
  node_ip: 172.16.23.251
  pid: 11314
  time_this_iter_s: 1.001068115234375
  time_total_s: 1.001068115234375
  timestamp: 1532476433
  timesteps_this_iter: 10
  timesteps_total: 10
  training_iteration: 1
  
TrainingResult for train_func_2_dummy=2:
  date: 2018-07-24_17-53-51
  done: true
  experiment_id: 9934a516f4384195af80074b01a2d403
  hostname: build
  mean_accuracy: 2
  node_ip: 172.16.23.251
  pid: 11236
  time_this_iter_s: 1.0011522769927979
  time_tota

In [ ]:
import